In [8]:
!pip install --upgrade pyqt5 lxml 

In [2]:
!git clone https://github.com/tzutalin/labelImg

Cloning into 'labelImg'...
remote: Enumerating objects: 2097, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 2097 (delta 0), reused 0 (delta 0), pack-reused 2093 (from 2)
Receiving objects: 100% (2097/2097), 237.14 MiB | 9.19 MiB/s, done.
Resolving deltas: 100% (1245/1245), done.


In [4]:
import os 
if os.name == 'posix':
    !cd labelImg &&  make qt5py3
if os.name =='nt':
    !cd labelImg && pyrcc5 -o libs/resources.py resources.qrc

pyrcc5 -o libs/resources.py resources.qrc


In [10]:
!cd labelImg && python labelImg.py

2025-06-20 08:35:15.099 python[30927:6665402] The class 'NSOpenPanel' overrides the method identifier.  This method is implemented by class 'NSWindow'
Image:/Users/yash/Library/IntelliShelf/defect-dataset/images/drum/defected-drum1.jpeg -> Annotation:/Users/yash/Library/IntelliShelf/defect-dataset/images/drum/defected-drum1.txt
[('defect', [(79, 39), (213, 39), (213, 81), (79, 81)], None, None, False), ('defect', [(30, 77), (79, 77), (79, 106), (30, 106)], None, None, False)]
Image:/Users/yash/Library/IntelliShelf/defect-dataset/images/drum/defected-drum2.jpeg -> Annotation:/Users/yash/Library/IntelliShelf/defect-dataset/images/drum/defected-drum2.txt
Image:/Users/yash/Library/IntelliShelf/defect-dataset/images/drum/defected-drum3.jpeg -> Annotation:/Users/yash/Library/IntelliShelf/defect-dataset/images/drum/defected-drum3.txt
[('defect', [(56, 58), (133, 58), (133, 104), (56, 104)], None, None, False), ('defect', [(130, 127), (152, 127), (152, 156), (130, 156)], None, None, False)]
Im

In [13]:
# yolo detect train \
#   model=yolov8n.pt \
#   data=defect.yaml \
#   imgsz=512 \
#   epochs=30 \
#   batch=8


In [24]:
from ultralytics import YOLO
from pathlib import Path

# ------------------------------------------------------------------
# 1. configure paths and confidence threshold
# ------------------------------------------------------------------
MODEL_PATH   = Path("/Users/yash/Library/IntelliShelf/runs/detect/train2/weights/best.pt")   # your trained weights
DEFECT_CLASS = 0                 # index you used for “defect” in defect.yaml
CONF_THRES   = 0.25              # treat predictions above this as valid

# ------------------------------------------------------------------
# 2. load model once
# ------------------------------------------------------------------
model = YOLO(MODEL_PATH)

def is_damaged(image_path: str | Path, conf_thresh: float = CONF_THRES) -> tuple[bool, float]:
    """
    Returns (damaged?, highest_defect_confidence)
    """
    results = model.predict(source=str(image_path), imgsz=512, conf=conf_thresh, verbose=False)
    boxes = results[0].boxes

    # find max confidence for defect class
    max_conf = 0.0
    for b in boxes:
        if int(b.cls[0]) == DEFECT_CLASS:
            max_conf = max(max_conf, float(b.conf[0]))

    damaged = max_conf >= conf_thresh
    return damaged, max_conf

# ------------------------------------------------------------------
# 3. test it
# ------------------------------------------------------------------
# img_path = "/Users/yash/Library/IntelliShelf/defect-dataset/images/train/defected-drum2.jpeg"   # replace with your image
img_path = "/Users/yash/Library/IntelliShelf/dataset/images/drum/0001.jpg"
damaged, conf = is_damaged(img_path)

print(f"Image: {img_path}")
if damaged:
    print(f"🚨 Defected (confidence {conf:.2f})")
else:
    print(f"✅ Clean (no defect ≥ {CONF_THRES})")



Image: /Users/yash/Library/IntelliShelf/dataset/images/drum/0001.jpg
✅ Clean (no defect ≥ 0.25)


In [25]:
# ------------------------------------------------------------------
# 3. test it
# ------------------------------------------------------------------
img_path = "/Users/yash/Library/IntelliShelf/defect-dataset/images/train/defected-drum2.jpeg"   # replace with your image
# img_path = "/Users/yash/Library/IntelliShelf/dataset/images/drum/0001.jpg"
damaged, conf = is_damaged(img_path)

print(f"Image: {img_path}")
if damaged:
    print(f"🚨 Ddefected (confidence {conf:.2f})")
else:
    print(f"✅ Clean (no defect ≥ {CONF_THRES})")

Image: /Users/yash/Library/IntelliShelf/defect-dataset/images/train/defected-drum2.jpeg
🚨 Ddefected (confidence 0.51)
